This file is focused on splitting the entire pbp file into weeks and individual games with the plays sorted in order from start to end.

In [ ]:
import pandas as pd
import re
import os

In [4]:
df = pd.read_csv('../../Data/pbp-2023.csv')
print(df.columns)

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'Unnamed: 10',
       'SeriesFirstDown', 'Unnamed: 12', 'NextScore', 'Description', 'TeamWin',
       'Unnamed: 16', 'Unnamed: 17', 'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsSack', 'IsChallenge', 'IsChallengeReversed',
       'Challenger', 'IsMeasurement', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards'],
      dtype='object')


In [5]:
dropped_columns = ['Unnamed: 10', 'Unnamed: 12', 'Unnamed: 16', 'Unnamed: 17', 'IsMeasurement']
df = df.drop(columns = dropped_columns)

In [6]:
teams = []

for team in df['OffenseTeam']:
    if team not in teams:
        teams.append(team)

In [7]:
dates = (df['GameDate'].unique())
dates = pd.to_datetime(dates)
dates = sorted(dates)

In [8]:
def get_week(dates):
    week_numbers = []
    week_num = 1

    # Initialize week_start to the Thursday before the first game
    week_start = dates[0] - pd.Timedelta(days=(dates[0].weekday() + 4) % 7)

    for date in dates:
        # Check if we need to start a new week
        if date - week_start >= pd.Timedelta(days=7):  
            week_num += 1
            week_start = date - pd.Timedelta(days=(date.weekday() + 4) % 7)  # Adjust to the new Thursday

        date_str = date.strftime('%Y-%m-%d')
        week_numbers.append((date_str, week_num))

    return week_numbers

In [9]:
dates_with_weeks = get_week(dates)

In [83]:
for i in range(1, 19):
    file_dir = os.path.join('../../Clean_Data/Weeks', str(i))
    os.makedirs(file_dir)

In [29]:
dir = '../../Clean_Data/Weeks'
def create_files(week, game_id_arr):
    week_dir = os.path.join(dir, week)

    if not os.path.exists(week_dir):
            os.makedirs(week_dir)

    for game_id in game_id_arr:
        file_path = os.path.join(week_dir, f"{game_id}.csv")
        with open(file_path, 'w') as f:
            pass 
        


In [38]:
for date in dates_with_weeks:
    print(date[0])

2023-09-07
2023-09-10
2023-09-11
2023-09-14
2023-09-17
2023-09-18
2023-09-21
2023-09-24
2023-09-25
2023-09-28
2023-10-01
2023-10-02
2023-10-05
2023-10-08
2023-10-09
2023-10-12
2023-10-15
2023-10-16
2023-10-19
2023-10-22
2023-10-23
2023-10-26
2023-10-29
2023-10-30
2023-11-02
2023-11-05
2023-11-06
2023-11-09
2023-11-12
2023-11-13
2023-11-16
2023-11-19
2023-11-20
2023-11-23
2023-11-24
2023-11-26
2023-11-27
2023-11-30
2023-12-03
2023-12-04
2023-12-07
2023-12-10
2023-12-11
2023-12-14
2023-12-16
2023-12-17
2023-12-18
2023-12-21
2023-12-23
2023-12-24
2023-12-25
2023-12-28
2023-12-30
2023-12-31
2024-01-06
2024-01-07


In [84]:
current_week = 1

game_ids = []

for date in dates_with_weeks:
    if date[1] is not current_week:
        create_files(str(current_week), game_ids)
        game_ids = []
        current_week += 1
       
    games_on_date = df[df['GameDate'] == date[0]]

    for _, row in games_on_date.iterrows():
        if row['GameId'] not in game_ids:
            game_ids.append(row['GameId'])

create_files(str(current_week), game_ids)


In [60]:
game_df = df[df['GameId'] == int(game_id)]

game_df.shape

(208, 40)

In [63]:
print(game_df.columns)

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'SeriesFirstDown',
       'NextScore', 'Description', 'TeamWin', 'SeasonYear', 'Yards',
       'Formation', 'PlayType', 'IsRush', 'IsPass', 'IsIncomplete',
       'IsTouchdown', 'PassType', 'IsSack', 'IsChallenge',
       'IsChallengeReversed', 'Challenger', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards'],
      dtype='object')


In [122]:
def calc_duration(play):
    # elapsed
    quarter_second = {
        1 : 0,
        2 : 900,
        3 : 1800,
        4 : 2700,
        5 : 3600, # OT
    }

    quarter_elapsed_seconds = 900 - ((play['Minute'] * 60) + play['Second'])
    
    elapsed_seconds = quarter_second.get(play['Quarter'], 0) + quarter_elapsed_seconds

    return elapsed_seconds

In [ ]:
directory = '../../Clean_Data/Weeks'

for week in os.listdir(directory):
    week_path = os.path.join(directory, week)

    for game in os.listdir(os.path.join(directory, week)):
        game_path = os.path.join(week_path, game)

        game_id = game[:-4] # remove file ext

        game_df = df[df['GameId'] == int(game_id)]
        
        game_df['Duration'] = game_df.apply(calc_duration, axis=1)


        offense_team = game_df.iloc[0]['OffenseTeam']
        defense_team = game_df.iloc[0]['DefenseTeam']

        game_df[f'{offense_team}'] = 0
        game_df[f'{defense_team}'] = 0

        game_df.sort_values(by='Duration', inplace=True)
        game_df.to_csv(game_path, index=False)

